# Pluvial flood risk

This example shows a workflow to derive pluvial flood risk using the **SFINCS** and **Delft-FIAT** models. The starting point is a user defined region and data catalog. Rainfall IDF curves from the GPEX dataset are translated into design events (hyetographs) for different return periods and used to simulate the flood hazard maps. The hazard maps are combined with exposure and impact data to derive risk.

In [ ]:
# Import packages and setup logging
from pathlib import Path

from hydroflows import Workflow, WorkflowConfig
from hydroflows.log import setuplog
from hydroflows.methods import fiat, flood_adapt, rainfall, sfincs
from hydroflows.utils.example_data import fetch_data

logger = setuplog(level="INFO")

In [ ]:
# General setup of workflow
name = "pluvial_risk"
pwd = Path().resolve()  # Get the current file location
case_root = Path(pwd, "cases", name)  # output directory
pwd_rel = "../../"  # relative path from the case directory to the current file

## Workflow inputs

The example requires the following inputs which are provided via a configuration file:
- a user defined region that can be used to delineate the SFINCS model domain
- a data catalog file describing all input datasets, see [HydroMT Core docs](https://deltares.github.io/hydromt/v0.10.0/user_guide/data_prepare_cat.html) 
- The [GPEX IDF dataset](https://data.4tu.nl/articles/dataset/GPEX_Global_Precipitation_EXtremes/12764429/4). Note that rainfall timeseries data can also be used to derive design events.
- HydroMT configuration files for both models, see [hydromt_sfincs docs](https://deltares.github.io/hydromt_sfincs/latest/) and [hydromt_fiat docs](https://deltares.github.io/hydromt_fiat/latest/)
- model executables (docker is also possible for SFINCS)

In [ ]:
# Fetch the test global-data data catalog  
cache_dir = fetch_data(data="global-data")

In [ ]:
# Setup the config file
config = WorkflowConfig(
    # general settings
    region=Path(pwd_rel, "data/build/region.geojson"),
    catalog_path=Path(cache_dir, "data_catalog.yml"),
    plot_fig=True,
    # sfincs settings
    hydromt_sfincs_config=Path(pwd_rel, "hydromt_config/sfincs_config.yml"),
    sfincs_exe=Path(pwd_rel, "bin/sfincs_v2.1.1/sfincs.exe"),
    # fiat settings
    hydromt_fiat_config=Path(pwd_rel, "hydromt_config/fiat_config.yml"),
    fiat_exe=Path(pwd_rel, "bin/fiat_v0.2.0/fiat.exe"),
    risk=True,
    # rainfall and design events settings
    start_date="2000-01-01",
    end_date="2021-12-31",
    rps=[2, 5, 10, 50, 100],
)


## Create the workflow

In [ ]:
# create and empty workflow
wf = Workflow(name=name, config=config, root=case_root)

### Build models

In [ ]:
# Build a SFINCS model for a given region using
# - setting from the hydromt_sfincs_config
# - data from the catalog_path
sfincs_build = sfincs.SfincsBuild(
    region=wf.get_ref("$config.region"),
    sfincs_root="models/sfincs",
    config=wf.get_ref("$config.hydromt_sfincs_config"),
    catalog_path=wf.get_ref("$config.catalog_path"),
    plot_fig=wf.get_ref("$config.plot_fig"),
    # save subgrid output to use in subsequent rules:
    subgrid_output=True, 
)
wf.create_rule(sfincs_build, rule_id="sfincs_build")

In [ ]:
# Build a FIAT model using 
# - the sfincs_build output for the model region and ground elevation
# - settings from the hydromt_fiat_config
# - data from the data catalog
fiat_build = fiat.FIATBuild(
    region=sfincs_build.output.sfincs_region,
    ground_elevation=sfincs_build.output.sfincs_subgrid_dep,
    fiat_root="models/fiat",
    catalog_path=wf.get_ref("$config.catalog_path"),
    config=wf.get_ref("$config.hydromt_fiat_config"),
)
wf.create_rule(fiat_build, rule_id="fiat_build")

### Derive pluvial design events

In [ ]:
# Define pluvial events from GPEX
# Alternatively, you can use the `rainfall.PluvialDesignEvents` class to define pluvial events from a rainfall time series
pluvial_events = rainfall.PluvialDesignEventsGPEX(
    gpex_nc=Path(cache_dir, "gpex.nc"),  
    region=sfincs_build.output.sfincs_region,
    event_root="data/events",
    rps=wf.get_ref("$config.rps"),
    wildcard="event", # wildcard to use for the pluvial events
)

# Note that a new "event" wildcard is created for the events
wf.create_rule(pluvial_events, rule_id="pluvial_events")


### Derive flood hazard

In [ ]:
# Update the SFINCS model with pluvial events
# This will create new SFINCS instances for each pluvial event in the simulations subfolder
sfincs_update = sfincs.SfincsUpdateForcing(
    sfincs_inp=sfincs_build.output.sfincs_inp,
    event_yaml=pluvial_events.output.event_yaml,
    output_dir=sfincs_build.output.sfincs_inp.parent/"simulations"
)
wf.create_rule(sfincs_update, rule_id="sfincs_update")


In [ ]:
# Run the SFINCS model for each pluvial event
# This will create simulated water levels for each pluvial event
sfincs_run = sfincs.SfincsRun(
    sfincs_inp=sfincs_update.output.sfincs_out_inp,
    sfincs_exe=wf.get_ref("$config.sfincs_exe"),
    run_method="exe", # alternatively use "docker" to run in a docker container
)
wf.create_rule(sfincs_run, rule_id="sfincs_run")


In [ ]:
# Postprocesses SFINCS results to a regular grid of maximum water levels
# To create high-resolution flood maps, you can use the `sfincs.SfincsDownscale` class
sfincs_post = sfincs.SfincsPostprocess(
    sfincs_map=sfincs_run.output.sfincs_map,
)
wf.create_rule(sfincs_post, rule_id="sfincs_post")


### Derive flood risk

In [ ]:
# Update FIAT hazard forcing with the pluvial eventset to compute pluvial flood risk
fiat_update = fiat.FIATUpdateHazard(
    fiat_cfg=fiat_build.output.fiat_cfg,
    event_set_yaml=pluvial_events.output.event_set_yaml,
    map_type="water_level",
    hazard_maps=sfincs_post.output.sfincs_zsmax,
    risk=wf.get_ref("$config.risk"),
    output_dir=fiat_build.output.fiat_cfg.parent/"simulations"
)
wf.create_rule(fiat_update, rule_id="fiat_update")

In [ ]:
# Run FIAT to compute pluvial flood risk
fiat_run = fiat.FIATRun(
    fiat_cfg=fiat_update.output.fiat_out_cfg,
    fiat_exe=wf.get_ref("$config.fiat_exe"),
)
wf.create_rule(fiat_run, rule_id="fiat_run")


In [ ]:
# Visualize Fiat 
fiat_visualize = fiat.FIATVisualize(
    fiat_cfg=fiat_update.output.fiat_out_cfg,
    fiat_output_csv=fiat_run.output.fiat_out_csv,
    spatial_joins_cfg=fiat_build.output.spatial_joins_cfg,
)
wf.create_rule(fiat_visualize, rule_id="fiat_visualize")

### Prepare FloodAdapt database input

In [ ]:
# prepare the FloodAdapt database builder  
fa_run = flood_adapt.SetupFloodAdapt(
    fiat_cfg=fiat_build.output.fiat_cfg,
    sfincs_inp=sfincs_build.output.sfincs_inp,
    event_set_yaml=pluvial_events.output.event_set_yaml,
)
wf.create_rule(fa_run, rule_id="setup_flood_adapt")

## Visualize and execute the workflow

The workflow can be executed using HydroFlows or a workflow engine. 
Below we first plot and dryrun the workflow to check if it is correctly defined. 
Then, we parse the workflow to `SnakeMake` to execute it

In [ ]:
# plot the rulegraph using graphviz
wf.plot_rulegraph(filename="rulegraph.svg", plot_rule_attrs=True)

In [ ]:
# dryrun workflow. Make sure no warnings are raised
# to run the workflow in HydroFlows use wf.run(), or (preferred) use wf.to_snakemake() to create a snakemake file
wf.dryrun()


In [ ]:
# Write the workflow to a Snakefile
wf.to_snakemake()

# show the files in the case directory
print(f"{wf.root.relative_to(pwd)}:")
for f in wf.root.iterdir():
    if f.is_file():
        print(f"- {f.name}")

In [ ]:
# uncomment to run the workflow with snakemake
# import subprocess
# subprocess.run(["snakemake", "-c", "1"], cwd=wf.root)